In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

from resources import gkey

gmaps.configure(api_key = gkey)

In [2]:
cities_data_df = pd.read_csv("output_data/cities.csv")
cities_data_df.head()

,City,Cloudiness,Contry,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Mar del Plata,90,AR,1585625566,88,-38.00,-57.56,64.99,13.87
1,Busselton,97,AU,1585625656,36,-33.65,115.33,75.00,4.45
2,Ushuaia,40,AR,1585625445,61,-54.80,-68.30,50.00,19.46
3,Taltal,55,CL,1585625656,69,-25.40,-70.48,65.97,0.94
4,Port Alfred,69,ZA,1585625656,97,-33.59,26.89,66.00,4.00


In [3]:
fig = gmaps.figure()
locations = cities_data_df[["Lat", "Lng"]]
weights = cities_data_df["Humidity"]
fig.add_layer(gmaps.heatmap_layer(locations, weights = weights))
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
candidates_df = cities_data_df.loc[(cities_data_df["Max Temp"] < 80) & (cities_data_df["Max Temp"] > 70) ,:]
candidates_df = cities_data_df.loc[(cities_data_df["Cloudiness"] == 0), :]
candidates_df = candidates_df.head()
candidates_df

,City,Cloudiness,Contry,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
17,Olonets,0,RU,1585625664,89,60.98,32.97,22.75,3.40
19,La Baule-Escoublac,0,FR,1585625664,80,47.29,-2.36,42.01,13.87
22,Saint-François,0,FR,1585625665,63,46.42,3.91,28.40,2.24
24,Kondopoga,0,RU,1585625666,72,62.21,34.26,15.80,4.47
32,Bāfq,0,IR,1585625668,24,31.61,55.41,59.23,1.48


In [5]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
locations = []
hotel_names = []
cities = []
countries = []
infoBoxes = []



for index, row in candidates_df.iterrows():
    
    params = {"location": str(row["Lat"]) + "," + str(row["Lng"]),
                "radius": 5000,
                "type": "hotel",
                "key": gkey}
                
    request = requests.get(url, params = params)
    
    response = request.json()
        
    try:
        location = (response["results"][0]["geometry"]["location"]["lat"],response["results"][0]["geometry"]["location"]["lng"])
        locations.append(location)
        hotel_names.append(response["results"][0]["name"])
        cities.append(row["City"])
        countries.append(row["Contry"])
    
        infoBoxes.append("Hotel name: " + response["results"][0]["name"] + ", City: " + row["City"] + ", Country: " + row["Contry"])

    except:
        print("No hotel found! Skipping...")
        
print(f"Locations: {str(len(locations))}")
print(f"Info Boxes: {str(len(infoBoxes))}")

#infoBoxes = "Hotel name:\n" + hotel_names + "\nCity: \n" + cities + "\nCountry: \n" + countries

markers = gmaps.marker_layer(locations, info_box_content = infoBoxes)
fig.add_layer(markers)
display(fig)

Locations: 5
Info Boxes: 5


Figure(layout=FigureLayout(height='420px'))